## Mapping

In [2]:
import torch
import os
import sys
base_dir = os.environ.get('TORCHSIM_DIR', default='/workspace/PyTorchSim')
sys.path.append(base_dir)

### Default mapping is based on heuristic.

In [3]:
from Scheduler.scheduler import PyTorchSimRunner
device = PyTorchSimRunner.setup_device().custom_device()

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

opt_fn = torch.compile(dynamic=False)(torch.matmul)
npu_out = opt_fn(input, weight)

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/npu/build.ninja...
Building extension module npu...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module npu...


ninja: no work to do.
Wrapper Codegen Path = /tmp/torchinductor_root/yr/cyrl4zqohiglmrez32dmaijhd3sfdh4xabea5splhxwtwckiykpr.py
[Gem5] Gem5 is running..  
[Spike] Running Spike simulator
[TOGSim] TOGSim is running.   
[TOGSim] Simulation of "/tmp/torchinductor/tmp/4q4qv6gbpia/tile_graph.onnx" is stored to "/tmp/torchinductor/tmp/4q4qv6gbpia/togsim_result/6"


In [4]:
!cat /tmp/torchinductor/tmp/4q4qv6gbpia/togsim_result/0 | grep "Total execution cycle"

[2025-11-30 18:53:14.002] [info] Total execution cycle: 47158


### Manual Mapping
User can set tile size manually.

In [5]:
torch._dynamo.reset()

os.environ['TORCHSIM_MANUAL_TILE_SIZE']="1"
os.environ['TORCHSIM_TILE_M']="512"
os.environ['TORCHSIM_TILE_N']="512"
os.environ['TORCHSIM_TILE_K']="512"

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

opt_fn = torch.compile(dynamic=False)(torch.matmul)
npu_out = opt_fn(input, weight)

Wrapper Codegen Path = /tmp/torchinductor_root/dg/cdgpndctsltydpvcgfyntgkgfmw6dp4a7vruwmjhxzilxzvyewby.py
[Gem5] Gem5 is running.   
[Spike] Running Spike simulator
[TOGSim] TOGSim is running... 
[TOGSim] Simulation of "/tmp/torchinductor/tmp/75hiq5mugpq/tile_graph.onnx" is stored to "/tmp/torchinductor/tmp/75hiq5mugpq/togsim_result/1"


In [6]:
!cat /tmp/torchinductor/tmp/75hiq5mugpq/togsim_result/0 | grep "Total execution cycle"

[2025-11-30 18:54:00.878] [info] Total execution cycle: 53704


### Autotune

In [7]:
torch._dynamo.reset()
os.environ['TORCHSIM_MANUAL_TILE_SIZE']="0"
os.environ['AUTOTUNE_TEMPLATE']="1"

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

opt_fn = torch.compile(dynamic=False)(torch.matmul)
npu_out = opt_fn(input, weight)

[Auto-tune] Trying tile size: [1024, 1024, 256, 128, 1024, 256]
[Auto-tune] Trying tile size: [256, 1024, 1024, 128, 1024, 1024]
[Auto-tune] Trying tile size: [1024, 256, 1024, 128, 256, 1024]
[Auto-tune] Trying tile size: [1024, 1024, 128, 128, 1024, 128]
[Auto-tune] Optimal tile size: [1024, 1024, 128, 128, 1024, 128], cycles: 46423
Wrapper Codegen Path = /tmp/torchinductor_root/xe/cxeovwxl3i2e2mljoiqq35wn6uzsksisgoqqx7pzrbwl3lj6um34.py
[Gem5] Gem5 is running..  
[Spike] Running Spike simulator
[TOGSim] TOGSim is running.   
[TOGSim] Simulation of "/tmp/torchinductor/tmp/x27ipc5avjg/tile_graph.onnx" is stored to "/tmp/torchinductor/tmp/x27ipc5avjg/togsim_result/3"


In [ ]:
!cat /tmp/torchinductor/tmp/x27ipc5avjg/togsim_result/1 | grep "Total execution cycle"